In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yfinance as yf
from datetime import datetime, timedelta

### Inputs importantes:

In [2]:
#Variáveis do modelo
buy_or_sell = 'buy' #'buy' or 'sell' setup 
data_type = 'adj_close' #'close' or 'adj_close'
start_date = '2010-02-01' #start date of backtest
start_date = datetime.strptime(start_date, '%Y-%m-%d')
start_date = start_date - pd.tseries.offsets.BDay(80)
eden_type = 'my' # 'my' == mm8 > mm80 // 'asc_mm' == mm 8 and 80 ascending // 'all' == both conditions
is_eden = 'sim' #'sim' ou 'nao'
target_type = 'fixed' # 'amp' or 'fixed'
multiplier = 1.61 # if target_type = 'fixed', Multiply the risk to calculate the target. Normally 1.61 or 2

#### Inserção dos Ativos

In [3]:
stocks_list = ["CIEL3","ITSA4", "ALUP11","SUZB3", "CCRO3", "BBSE3", "JBSS3", "ENBR3", "MRVE3", "ASAI3", "CRFB3", "TOTS3", "GGBR4", "TIMS3", "IRBR3", "USIM5", "BRFS3", "BBDC4", "BRAP4", "ABEV3", "UGPA3", "PETR3", "VBBR3", "BPAC11", "ITSA4", "BPAN4", "CSAN3", "DXCO3", "TAEE11", "PCAR3", "CIEL3", "RENT3", "PRIO3", "EGIE3", "HAPV3", "LWSA3", "LREN3", "FLRY3", "CPLE6", "RRRP3", "ALPA4", "CASH3", "COGN3", "ENGI11", "VALE3", "PETZ3", "SUZB3", "EMBR3", "MRFG3", "KLBN11", "AZUL4", "CMIN3", "AMER3", "ELET6", "BRKM5", "BEEF3", "CSNA3", "MGLU3", "SLCE3", "PETR4", "POSI3", "GOLL4", "SBSP3", "CYRE3", "EQTL3", "RDOR3", "GOAU4", "WEGE3", "ECOR3", "ARZZ3", "NTCO3", "CPFE3", "IGTI11", "EZTC3", "B3SA3", "CMIG4", "RAIL3", "RAIZ4", "HYPE3", "ITUB4", "ENEV3", "BBAS3", "VIIA3", "CVCB3", "SOMA3", "VIVT3", "MULT3", "SMTO3", "YDUQ3", "SANB11"]
stocks_list = [i + '.SA' for i in stocks_list]

In [4]:
# Validar tickers descontinuados
if len(stocks_list) > 1:
    ticker_valid = yf.download(stocks_list, start = '2024-01-01', end = '2024-01-31', period = "1d")
    ticker_valid = ticker_valid['Close']
    ticker_valid.dropna(axis = 1, inplace=True)
    stocks_list = ticker_valid.columns

[*********************100%***********************]  87 of 87 completed

2 Failed downloads:
- VIIA3.SA: No data found, symbol may be delisted
- ENBR3.SA: No data found, symbol may be delisted


# Funções Auxiliares

### Definindo o algoritmo para simular as operações

In [5]:
import math

# Create a function to round any number to the smalles multiple of 100
def round_down(x):
    return int(math.floor(x / 100.0)) * 100

def backtest_algorithm_buy(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False
    for i in range(0,len(df)):
        if ongoing == True:
            if (df["Open"][i] >= target) | (df["Open"][i] <= stop): 
                exit = df["Open"][i]

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= stop: 
                exit = stop

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["High"][i] >= target: 
                exit = target

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

        else:
 
            if ~(np.isnan(df["buy_price"][i])):
                entry = df["buy_price"][i]
                stop = df["stop"][i]
                
                if df["Low"][i] > stop: 
                    ongoing = True
                    risk = entry - stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

def backtest_algorithm_sell(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False
    for i in range(0,len(df)):
        if ongoing == True:
            if (df["Open"][i] <= target) | (df["Open"][i] >= stop): 
                exit = df["Open"][i]

                profit = shares * (entry - exit)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False
                
            elif df["High"][i] >= stop: 
                exit = stop

                profit = shares * (entry - exit)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= target: 
                exit = target

                profit = shares * (entry - exit)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False
            print(total_capital)


        else:
            if ~(np.isnan(df["sell_price"][i])):
                entry = df["sell_price"][i]
                stop = df["stop"][i]
                
                if df["High"][i] > stop: 
                    ongoing = True
                    risk = entry + stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

### Calculando a estatística e a curva de capital

In [6]:
def get_drawdown(data, column = "Close"):
    data["Max"] = data[column].cummax()
    data["Delta"] = data['Max'] - data[column]
    data["Drawdown"] = 100 * (data["Delta"] / data["Max"])
    max_drawdown = data["Drawdown"].max()
    return max_drawdown

def strategy_test(all_profits, total_capital):
    if len(all_profits) > 0:
        gains = sum(x >= 0 for x in all_profits)
        losses = sum(x < 0 for x in all_profits)
        num_operations = gains + losses
        pct_gains = 100 * (gains / num_operations)
        pct_losses = 100 - pct_gains
        total_profit = sum(all_profits)
        pct_profit = (total_profit / total_capital[0]) * 100
        
        # Compute drawdown
        total_capital = pd.DataFrame(data=total_capital, columns=["total_capital"])
        drawdown = get_drawdown(data=total_capital, column="total_capital")

        # Compute profit per operation
        profit_per_operation = pct_profit / num_operations

        # Expeceted Value
        all_positives = [x for x in all_profits if x >= 0]
        if len(all_positives) > 0:
            average_gain = sum(all_positives) / len(all_positives)
        else:
            average_gain = 0

        all_negatives = [x for x in all_profits if x < 0]
        if len(all_negatives) > 0:
            average_loss = sum(all_negatives) / len(all_negatives)
        else:
            average_loss = 0
            
        num_operations = len(all_profits)
        pct_gains = (len(all_positives) / num_operations)
        pct_losses = 1 - pct_gains
        
        expected_value = (average_gain * pct_gains) + (average_loss * pct_losses)
    else:
        total_profit = 0
        pct_profit = 0
        drawdown = 0
        num_operations = 0
        profit_per_operation = 0
        gains = 0
        pct_gains = 0
        losses = 0
        pct_losses = 0
        expected_value = 0

    return {
        "total_profit": total_profit,
        "pct_profit": pct_profit,
        "pct_drawdown": drawdown,
        "num_operations": num_operations,
        "pct_profit_per_operation": profit_per_operation,
        "num_gains": gains ,
        "pct_gains": pct_gains,
        "num_losses": losses,
        "pct_losses": pct_losses, 
        "expected_value": expected_value
    }
def capital_plot(total_capital, all_profits):
  all_profits = [0] + all_profits # make sure both lists are the same size
  cap_evolution = pd.DataFrame({'Capital': total_capital, 'Profit': all_profits})
  plt.title("Curva de Capital")
  plt.xlabel("Total Operações")
  cap_evolution['Capital'].plot()

# Ajuste do DF para chamada das Funções de modo Escalável

## 123 de Compra

In [7]:
if buy_or_sell == 'buy':
    df_stats = pd.DataFrame()
    for stock in stocks_list:
        df = yf.download(stock, start = start_date, interval = '1d')
        if data_type == 'adj_close':
            df.drop(['Close','Volume'], axis=1, inplace=True)
            df.rename({"Adj Close":"Close"},inplace=True,axis=1)
        else:
            df.drop(['Adj Close','Volume'], axis=1, inplace=True)

    ### Definindo o candle sinal
        condition_1 = df["Low"] > df["Low"].shift(1)
        condition_2 = df["Low"].shift(1) < df["Low"].shift(2)
        df["signal"] = condition_1 & condition_2

    ### Calculando o Éden dos Traders
        df["mme8"] = df["Close"].rolling(window=8).mean()
        df["mme80"] = df["Close"].rolling(window=80).mean()

        if eden_type == 'my':
            df["eden"] = (df["mme8"] > df["mme80"])
        elif eden_type == 'asc_mms':
            df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1))
        else:
            df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1)) & (df["mme8"] > df["mme80"])
        df.dropna(inplace=True)

    ### Definindo o preço de compra
        condition_1 = df["signal"].shift(1) == True
        condition_2 = df["eden"].shift(1) == True 
        condition_3 = df["High"] > df["High"].shift(1)
        tick = 0.01

        if is_eden == 'sim':
            df["buy_price"] = np.where(
            condition_1 & condition_2 & condition_3, 
            np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
            np.nan)
        else:
            df["buy_price"] = np.where(
            condition_1 & condition_3, 
            np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
            np.nan)
        
    ### Definindo o alvo
        max_high = df["High"].rolling(3).max()
        min_low = df["Low"].rolling(3).min()
        amplitude = (max_high.shift(1) - min_low.shift(1))
        fixed_risk = ((df["High"].shift(1) +tick) - (df["Low"].shift(2) - tick)) * multiplier
        entry = (df["High"].shift(1)) + tick 
        if target_type == 'fixed':
            df["target"] = fixed_risk + entry
        else:
            df["target"] =  amplitude + entry

    ### Definindo o stop
        df["stop"] = df["Low"].shift(2) - tick

    ### Realizando o Backtest
        all_profits, total_capital = backtest_algorithm_buy(
        df=df,
        capital_exposure=1000,
        initial_capital=100000)
        statistics = strategy_test(all_profits, total_capital)
        statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
        df_stats[stock] = statistics
    df_stats = df_stats.T

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## 123 de Venda

In [8]:
if buy_or_sell == 'sell':
    df_stats = pd.DataFrame()
    for stock in stocks_list:
        df = yf.download(stock, start = start_date, interval = '1d')
        if data_type == 'adj_close':
            df.drop(['Close','Volume'], axis=1, inplace=True)
            df.rename({"Adj Close":"Close"},inplace=True,axis=1)
        else:
            df.drop(['Adj Close','Volume'], axis=1, inplace=True)

    ### Definindo o candle sinal
        condition_1 = df["High"] < df["High"].shift(1)
        condition_2 = df["High"].shift(1) > df["High"].shift(2)
        df["signal"] = condition_1 & condition_2

    ### Calculando o Éden dos Traders
        df["mme8"] = df["Close"].rolling(window=8).mean()
        df["mme80"] = df["Close"].rolling(window=80).mean()

        if eden_type == 'my':
            df["eden"] = (df["mme8"] < df["mme80"])
        elif eden_type == 'asc_mms':
            df["eden"] = (df["mme8"] < df["mme8"].shift(1)) & (df["mme80"] < df["mme80"].shift(1))
        else:
            df["eden"] = (df["mme8"] < df["mme8"].shift(1)) & (df["mme80"] < df["mme80"].shift(1)) & (df["mme8"] < df["mme80"])
        df.dropna(inplace=True)

    ### Definindo o preço de VENDA
        condition_1 = df["signal"].shift(1) == True
        condition_2 = df["eden"].shift(1) == True 
        condition_3 = df["Low"] < df["Low"].shift(1)
        tick = 0.01

        if is_eden == 'sim':
            df["sell_price"] = np.where(
            condition_1 & condition_2 & condition_3, 
            np.where(df["Open"] < df["Low"].shift(1), df["Open"], df["Low"].shift(1) - tick),
            np.nan)
        else:
            df["sell_price"] = np.where(
            condition_1 & condition_3, 
            np.where(df["Open"] < df["Low"].shift(1), df["Open"], df["Low"].shift(1) - tick),
            np.nan)
        
    ### Definindo o alvo
        max_high = df["High"].rolling(3).max()
        min_low = df["Low"].rolling(3).min()
        amplitude = (max_high.shift(1) - min_low.shift(1))
        fixed_risk = ((df["High"].shift(2) +tick) - (df["Low"].shift(1) - tick)) * multiplier
        entry = (df["Low"].shift(1)) - tick 
        if target_type == 'fixed':
            df["target"] = entry - fixed_risk
        else:
            df["target"] =  entry - amplitude
    ### Definindo o stop
        df["stop"] = df["High"].shift(2) + tick

    ### Realizando o Backtest
        all_profits, total_capital = backtest_algorithm_sell(
        df=df,
        capital_exposure=1000,
        initial_capital=100000)
        statistics = strategy_test(all_profits, total_capital)
        statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
        df_stats[stock] = statistics
    df_stats = df_stats.T

In [9]:
df

,Open,High,Low,Close,signal,mme8,mme80,eden,buy_price,target,stop
Date,,,,,,,,,,,
2010-02-09,6.843333,7.023333,6.843333,4.876251,True,5.077003,5.374641,False,NaN,NaN,NaN
2010-02-10,7.000000,7.216666,7.000000,4.946078,False,5.040344,5.373535,False,NaN,NaN,NaN
2010-02-11,6.933333,7.200000,6.933333,4.960042,False,4.985355,5.369200,False,NaN,7.859933,6.833333
2010-02-12,6.893333,7.143333,6.893333,4.962370,False,4.942295,5.363585,False,NaN,7.564200,6.990000
2010-02-17,7.000000,7.383333,7.000000,5.155559,True,4.923384,5.361053,False,NaN,7.523633,6.923333
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-08,21.660000,21.930000,21.139999,21.330000,False,20.677500,20.500625,True,NaN,23.569800,20.490000
2024-03-11,21.260000,21.400000,20.830000,21.170000,False,20.785000,20.518000,True,NaN,24.049102,20.629999
2024-03-12,21.320000,21.469999,20.799999,21.270000,False,20.898750,20.534500,True,NaN,21.860800,21.129999


In [10]:
df_stats.sort_values('expected_value').tail(20)

,total_profit,pct_profit,pct_drawdown,num_operations,pct_profit_per_operation,num_gains,pct_gains,num_losses,pct_losses,expected_value
RAIL3.SA,12458.16,12.46,8.47,79.0,0.16,36.0,0.46,43.0,0.54,157.70
ELET6.SA,16373.29,16.37,4.49,101.0,0.16,48.0,0.48,53.0,0.52,162.11
UGPA3.SA,18258.54,18.26,11.12,111.0,0.16,53.0,0.48,58.0,0.52,164.49
ALPA4.SA,19818.81,19.82,7.49,118.0,0.17,55.0,0.47,63.0,0.53,167.96
POSI3.SA,12181.16,12.18,10.84,72.0,0.17,31.0,0.43,41.0,0.57,169.18
SLCE3.SA,17551.26,17.55,16.83,100.0,0.18,46.0,0.46,54.0,0.54,175.51
EZTC3.SA,23314.72,23.31,11.60,122.0,0.19,57.0,0.47,65.0,0.53,191.10
GOLL4.SA,20067.06,20.07,5.26,105.0,0.19,51.0,0.49,54.0,0.51,191.11
BRKM5.SA,18693.51,18.69,5.92,97.0,0.19,45.0,0.46,52.0,0.54,192.72
LWSA3.SA,4583.72,4.58,4.50,21.0,0.22,10.0,0.48,11.0,0.52,218.27


# Mlhorias a serem feitas

- Adicionar filtro Ibov
- Alterar modelo de backtest
- Adicionar filtro Gap
- Adicionar filtro amplitude (se a amp entre o preço de entrada e o stop forem muito longos)
- Adicionar stop por tempo
- Adicionar filtro de fechamento dos 3 candles acima da mm8 (compra)